In [ ]:
import sys
# setting path
sys.path.append('..')
# importing
import ECC_library
from qiskit.circuit import Gate
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit import Aer, execute, IBMQ, BasicAer
from qiskit.aqua import QuantumInstance
from qiskit import IBMQ

In [ ]:
def test(a,b,p,n,g_s):
    x = QuantumRegister(n,name='x')
    y = QuantumRegister(n,name='y')
    c = QuantumRegister(1,name='d')
    z = QuantumRegister(n,name='z')
    g = QuantumRegister(1,name='g')
    m = QuantumRegister(n,name='m')
    r = QuantumRegister(n,name='r')
    clas_a = ClassicalRegister(n)
    clas_b = ClassicalRegister(n)
    clas_c = ClassicalRegister(1)
    clas_z = ClassicalRegister(n)
    clas_g = ClassicalRegister(1)
    clas_m = ClassicalRegister(n)
    clas_r = ClassicalRegister(n)
    circuit = QuantumCircuit(x,y,c,z,g,m,r,clas_a,clas_b,clas_c,clas_z,clas_g,clas_m,clas_r)
    s_a=bin(int(a))[2:].zfill(n)
    s_b=bin(int(b))[2:].zfill(n)
    s_p=bin(int(p))[2:].zfill(n)
    s_p=s_p[::-1]
    s_p1=s_p+'0'
    if(g_s=='1'):
        circuit.x(g)
    for i in range(0, n):
        if s_a[i] == '1':
            circuit.x(x[n-1-i])
        if s_b[i] == '1':
            circuit.x(y[n-1-i])

    sub_x = QuantumRegister(n)
    sub_y = QuantumRegister(n)
    sub_c = QuantumRegister(1)
    sub_z = QuantumRegister(n)
    sub_g = QuantumRegister(1)
    sub_m = QuantumRegister(n)
    sub_circuit = QuantumCircuit(sub_x,sub_y,sub_c,sub_z,sub_g,sub_m, name='mul_mod_p')
    for i in range (0,n):
        ECC_library.controlled_addition(sub_circuit,sub_x[i:i+1],sub_y,sub_z,sub_c,n)
        sub_circuit.cx(sub_z[0],sub_m[i])
        ECC_library.controlled_constant_addition(sub_circuit,sub_m[i:i+1],sub_z[0:n]+sub_c[0:1],sub_g,s_p1,n+1)
        ECC_library.binary_halving_cyclic(sub_circuit,sub_z[0:n]+sub_c[0:1],n+1)
    ECC_library.constant_subtraction(sub_circuit,sub_z[0:n]+sub_c[0:1],sub_g,s_p1,n+1)
    ECC_library.controlled_constant_addition(sub_circuit,sub_c,sub_z,sub_g,s_p,n)
    sub_inst = sub_circuit.to_gate()
    circuit.append(sub_inst,x[0:n]+y[0:n]+c[0:1]+z[0:n]+g[0:1]+m[0:n])

    for i in range (0,n):
        circuit.cx(z[i],r[i])

    circuit.append(sub_inst.inverse(),x[0:n]+y[0:n]+c[0:1]+z[0:n]+g[0:1]+m[0:n])

    for i in range(0, n):
        circuit.measure(x[i],clas_a[i])
    for i in range(0, n):
        circuit.measure(y[i],clas_b[i])
    circuit.measure(c,clas_c)
    for i in range(0, n):
        circuit.measure(z[i],clas_z[i])
    circuit.measure(g,clas_g)
    for i in range(0, n):
        circuit.measure(m[i],clas_m[i])
    for i in range(0, n):
        circuit.measure(r[i],clas_r[i])
    provider = IBMQ.get_provider('ibm-q')
    backend = provider.get_backend('ibmq_qasm_simulator')
    simulation = execute(circuit, backend=backend ,shots=10)
    sim_result = simulation.result()
    out=list(sim_result.get_counts().keys())[0]
    print('a:',a,'b:',b,'R:',(2**n)%p,'R_dag:',4,'p:',p,'n:',n,'g_s:',g_s)
    output=out.split(' ')
    output_a = output[6]
    output_b = output[5]
    res_c = output[4]
    output_z = output[3]
    res_g = output[2]
    output_m = output[1]
    output_r = output[0]
    res_a = int(output_a, 2)
    res_b = int(output_b, 2)
    res_z = int(output_z, 2)
    res_m = int(output_m, 2)
    res_r = int(output_r, 2)
    print('res_a:',res_a,'res_b:',res_b,'res_c:',res_c,'res_z:',res_z,'res_g:',res_g,'res_m:',res_m,'res_r:',res_r)
    if res_r!=(a*b*4)%p:
        print('error!')

In [ ]:
IBMQ.load_account();
for n in range (4,14):
    print('n:',n)
    d_vec=[]
    for p in [7,11,13,17,19,23,29,31,43,47]:
        if(p<2**n):
            print('p:',p)
            a=2
            b=3
            d=test(a,b,p,n,'0')
            d_vec.append(d)
            #test(a,b,p,n,'1')
    print(d_vec)
    print('--------')
